
## Task 1: CREDIT SCORING MODEL
## CODEALPHA | Machine Learning Intern | January Batch 2024

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

### Importing dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data = pd.read_excel("/content/drive/My Drive/creditscoredata.xlsx")

1st 5 Rows

In [7]:
data.head()

,TARGET,ID,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1,582,3,3,0,4,0.0,5,117,27,...,3.0,0.9179,0.2083,2,3,7,0.2083,4,4,0.0000
1,1,662,15,9,0,3,1.0,3,14,14,...,1.0,0.8000,0.0000,0,0,0,1.0000,12,0,1.0000
2,1,805,0,0,0,1,5.0,1,354,7,...,5.0,0.3552,0.6538,0,1,1,0.7308,1,1,0.5263
3,1,1175,8,5,0,6,1.0,10,16,4,...,3.0,0.9127,0.2500,1,1,1,0.7500,7,1,1.3333
4,1,1373,3,1,0,9,0.0,8,130,52,...,1.0,1.2511,0.0000,0,1,4,0.1429,3,1,0.0000


Shape of the data

In [6]:
data.shape

(3000, 30)

Dropping customer ID column from the dataset

In [8]:
data=data.drop('ID',axis=1)
data.shape

(3000, 29)

Explore missing values

In [9]:
data.isna().sum()

TARGET               0
DerogCnt             0
CollectCnt           0
BanruptcyInd         0
InqCnt06             0
InqTimeLast        188
InqFinanceCnt24      0
TLTimeFirst          0
TLTimeLast           0
TLCnt03              0
TLCnt12              0
TLCnt24              0
TLCnt                3
TLSum               40
TLMaxSum            40
TLSatCnt             4
TLDel60Cnt           0
TLBadCnt24           0
TL75UtilCnt         99
TL50UtilCnt         99
TLBalHCPct          41
TLSatPct             4
TLDel3060Cnt24       0
TLDel90Cnt24         0
TLDel60CntAll        0
TLOpenPct            3
TLBadDerogCnt        0
TLDel60Cnt24         0
TLOpen24Pct          3
dtype: int64

Filling missing values with mean

In [10]:
data=data.fillna(data.mean())

Explore missing values post missing value fix

In [11]:
data.isna().sum()

TARGET             0
DerogCnt           0
CollectCnt         0
BanruptcyInd       0
InqCnt06           0
InqTimeLast        0
InqFinanceCnt24    0
TLTimeFirst        0
TLTimeLast         0
TLCnt03            0
TLCnt12            0
TLCnt24            0
TLCnt              0
TLSum              0
TLMaxSum           0
TLSatCnt           0
TLDel60Cnt         0
TLBadCnt24         0
TL75UtilCnt        0
TL50UtilCnt        0
TLBalHCPct         0
TLSatPct           0
TLDel3060Cnt24     0
TLDel90Cnt24       0
TLDel60CntAll      0
TLOpenPct          0
TLBadDerogCnt      0
TLDel60Cnt24       0
TLOpen24Pct        0
dtype: int64

### Train Test Split

In [12]:
y = data.iloc[:, 0].values
X = data.iloc[:, 1:29].values

Splitting dataset into training and test (in ratio 80:20)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=y)

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Exporting Normalisation Coefficients for later use in prediction

In [15]:
import joblib
joblib.dump(sc, '/content/drive/My Drive/f2NormalizationcreditScore')

['/content/drive/My Drive/f2NormalizationcreditScore']

Risk Model building

In [16]:
classifier =  LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

Exporting Logistic Regression Classifier for later use in prediction

In [17]:
joblib.dump(classifier, '/content/drive/My Drive/f1ClassifiercreditScore')

['/content/drive/My Drive/f1ClassifiercreditScore']

Model performance

In [18]:
print(confusion_matrix(y_test,y_pred))

[[487  13]
 [ 87  13]]


In [19]:
print(accuracy_score(y_test, y_pred))

0.8333333333333334


Writing Output File

In [20]:
predictions = classifier.predict_proba(X_test)
predictions

array([[0.61644691, 0.38355309],
       [0.9885656 , 0.0114344 ],
       [0.87069686, 0.12930314],
       ...,
       [0.94450568, 0.05549432],
       [0.46756903, 0.53243097],
       [0.94014209, 0.05985791]])

Writing model output file


In [21]:
df_prediction_prob = pd.DataFrame(predictions, columns = ['prob_0', 'prob_1'])
df_prediction_target = pd.DataFrame(classifier.predict(X_test), columns = ['predicted_TARGET'])
df_test_dataset = pd.DataFrame(y_test,columns= ['Actual Outcome'])

dfx=pd.concat([df_test_dataset, df_prediction_prob, df_prediction_target], axis=1)

dfx.to_csv("/content/drive/My Drive/c1ModelPrediction.xlsx", sep=',', encoding='UTF-8')

dfx.head()


,Actual Outcome,prob_0,prob_1,predicted_TARGET
0,1,0.616447,0.383553,0
1,0,0.988566,0.011434,0
2,1,0.870697,0.129303,0
3,0,0.953963,0.046037,0
4,1,0.726633,0.273367,0


from matplotlib import pyplot as plt
_df_0['Actual Outcome'].plot(kind='hist', bins=20, title='Actual Outcome')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['prob_0'].plot(kind='hist', bins=20, title='prob_0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['prob_1'].plot(kind='hist', bins=20, title='prob_1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='Actual Outcome', y='prob_0', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['predicted_TARGET']
  ys = series['prob_1']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('predicted_TARGET', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('predicted_TARGET')
_ = plt.ylabel('prob_1')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['predicted_TARGET']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'predicted_TARGET'}, axis=1)
              .sort_values('predicted_TARGET', ascending=True))
  xs = counted['predicted_TARGET']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('predicted_TARGET', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('predicted_TARGET')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_9['Actual Outcome'].plot(kind='line', figsize=(8, 4), title='Actual Outcome')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['prob_0'].plot(kind='line', figsize=(8, 4), title='prob_0')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['prob_1'].plot(kind='line', figsize=(8, 4), title='prob_1')
plt.gca().spines[['top', 'right']].set_visible(False)